# Generador de retroalimentación para su posterior revisión por el docente
Este generador está pensado para pruebas de programación en las que el estudiante sube un solo fichero con su solución. Ejemplos en los que se pide que se implemente una librería, o un parte de un sistema. 

El generador va a iterar sobre todas las soluciones y va a generar un fichero HTML con el mismo nombre que el fichero de la prueba. Este es el primer paso de la coevaluación, en el segundo paso, el docente revisa y modifica la retroalimentación y calificación para poder notificarlo al estudiante.

Se espera que tengas dos carpetas preparadas para ser usadas:

1.- Carpeta **EspacioTrabajo** que contendrá el proyecto completo. En él posteriormente se copiaran la prueba del estudiante una a una y se compilará. Esta carpeta puede ser el esqueleto que se suele dar al estudiante para que trabaje durante el examen.

2.- Carpeta **pruebas** que contendrá todas las pruebas que necesitan retroalimentación. Se asume que es un sólo fichero y que vendrá con el nombre o id de cada estudiante en el nombre. Moodle tiene la opción de descargar todas las pruebas de una tarea.

In [ ]:
# Importamos las liberarías necesarias
import os #Para copiar el código del estudiante al proyecto para compilar
import subprocess #Para compilar el código
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
# Añadimos la clave de la API, has debido crearla en OpenIA y almacenarla en el .env del entorno Anaconda.
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL_GPT = 'o1-mini'
openai = OpenAI()


## Esta parte desglosa el promtp que se va a generar
* **Rúbrica general**: Contendrá las competencias que el docente quiere trabajar durante toda la asignatura (mantenibilidad, eficiencia, funcionalidad, etc).
* **Rúbrica específica**: Desglosará los métodos/funciones, su descripción y sus pesos sobre la calificación de la prueba.
* **Ejemplo de salida**: Tabla HTML que muestra un ejemplo de salida. Es interesante añadir los comentarios de retroalimentación que se esperan, y los que por experiencia, suele ser dónde fallan.

In [ ]:
# Rúbrica general
def rubricaGeneral():
    return """
Eres un evaluador de código de un examen de programación. Tu tarea es analizar la implementación del estudiante y generar una evaluación detallada que incluya una calificación numérica y una retroalimentación formativa para cada método. Para la retroalimentación ten en cuenta:
1. **Mantenimiento:**
 - Evaluar que el código sea fácilmente mantenible y extensible en el futuro.
 - Evitar el uso excesivo de bucles anidados o condiciones complejas.
 - Evitar funciones demasiado largas o complejas.
 - Evitar la duplicación de código.
 - Evitar la creación de variables locales innecesarias.
2. **Claridad:**
 - Asegurar que la estructura del código permita una lectura clara. 
 - Utilizar nombres descriptivos y claros en las variables.
 - Añadir comentarios para documentar partes complejas.
3. **Eficiencia:**
- Evitar llamadas recursivas innecesarias.
- Evitar recorridos innecesarios en estructuras de datos. 
4. **Funcionalidad:**
 - El código debe funcionar en llamadas correctas sin errores.
 - Cuando los parámetros de entrada no estén restringidos, el código de gestionar correctamente la introducción de parámetros incorrectos (por ejemplo valores NULL, número negativos, etc)
**Para la calificación de cada método**. Si la funcionalidad es correcta (punto 4 anterior), tendrá un 100% de la nota del método, si no, tendrá un máximo de un 50%. Puedes restar según los siguientes criterios hasta:
 - Un 10% si el código no es mantenible (punto 1 anterior)
 - Un 10% si el código no es claro (punto 2 anterior)
 - Un 5% si el código no es eficiente (punto 3 anterior)
**Formato de Salida:** La salida de la evaluación debe ser en HTML y contendrá:
 - Una línea con la calificación final sobre 10 y añade <br> al final.
 - Una tabla HTML con tres columnas: "Método", "Puntuación" y "Feedback".
 - Cada fila de la tabla debe corresponder a uno de los ítems evaluados
"""


In [ ]:
def rubricaEspecifica():
    return """Estos son los métodos a evaluar y dar una retroalimentación formativa:
 - `void poly_crear(struct Polinomio **p)` (1 punto): Esta función se encarga de crear un polinomio, reservando la memoria necesaria para la estructura que lo representa y debe asignar a *p la memoria incializada. Inicializa los valores de la estructura para que esté listo para su uso posterior. En caso de no poder pedir memoria, debe finalizar el programa con código -1.
 - `int poly_grado(const struct Polinomio *p)` (1 punto): Obtiene el grado de un polinomio. Devuelve -1 en caso de no poder calcularlo.
 - `int poly_coeficiente(const struct Polinomio *p, int exp)` (1 punto): Obtiene el coeficiente de un polinomio. Devuelve el coeficiente del monomio, o 0 si no existe, puede ser negativo (-3x^2).
 - `int poly_agregar(struct Polinomio *p, int coef, int exp)`(2.5 puntos): Inserta un monomio en un polinomio. En caso de existir suma (ojo que coef puede ser negativo o positivo, si queda a cero se elimina). En caso de no poder pedir memoria, debe finalizar el programa con código -1. En caso de no poder agregar, devuelve -1 en caso de no poder agregar. Devuelve 0 si puede agregar
 - `void poly_imprimir(const struct Polinomio *p)` (1.25 puntos): Imprime los coeficientes y exponentes de un polinomio. Esta función toma un puntero a una estructura Polinomio y muestra en la salida estándar los coeficientes y exponentes del polinomio en un formato legible. Un ejemplo de salida:3x^2 2x^4 4x^5 .Otro ejemplo cuando el polinomio no existe: Polinomio no existe.
 - `void poly_destruir(struct Polinomio *p)`(1.25 puntos): Destruye un polinomio liberando la memoria que ocupaba y dejando la estructura inicializada para alojar otro polinomio. Esta función se encarga de liberar la memoria que ocupaba un polinomio previamente inicializado. Se encarga de liberar la memoria de cada monomio que lo compone. Deja la estrutura struct Polinomio lista ser usada de nuevo.
 - `struct Polinomio *poly_sumar(const struct Polinomio *p1, const struct Polinomio *p2)` (2 puntos): Esta función toma dos estructuras de polinomios como entrada y devuelve una NUEVA estructura de polinomio que representa la suma de los dos polinomios de entrada. En caso de no poder realizar la suma, se debe devolver NULL. En caso de no poder pedir memoria, debe finalizar el programa con código -1.

 Esta es la estructura de datos:
 
struct Monomio
{
	int coef;
	int exp;
	struct Monomio *sig;
};

struct Polinomio
{
	struct Monomio *primero;
	struct Monomio *ultimo;
};"""

In [ ]:
def ejemploSalida():
    return """
 ** Ejemplo de salida **:
Calificación: 1/10

<table border="1">
  <tr>
    <th>Método</th>
    <th>Puntuación</th>
    <th>Retroalimentación</th>
  </tr>
  <tr>
    <td>poly_crear</td>
    <td>0.5 / 1</td>
    <td>
      <strong>Funcionalidad:</strong> La función no inicializa correctamente los punteros <code>primero</code> y <code>ultimo</code> después de reservar memoria. <br>
      <strong>Mantenimiento:</strong> Sin comentarios.<br>
      <strong>Claridad:</strong> Se crea una variable <code>aux</code> para pedir memoria y luego se asigna a la variable <code>*p</code>: usa directamente la variable <code>*p</code> y renombra a algo más descriptivo como <code>*polinomio</code>. Esto mejora la claridad del código con nombres descriptivos y elimina el uso de variables innecesarias que complica la compresión del código.<br>
      <strong>Eficiencia:</strong> Sin comentarios.<br>
      <em>Sugerencia Formativa:</em> Asegúrate de inicializar todos los campos de la estructura después de la asignación y utiliza <code>exit(-1)</code> para errores críticos en la reserva de memoria.
    </td>
  </tr>

  <tr>
    <td>poly_grado</td>
    <td>0.5 / 1</td>
    <td>
      <strong>Funcionalidad:</strong> El cálculo del grado del polinomio falla en casos en que el polinomio es nulo o la lista está vacía. No se contemplan condiciones de error, lo que puede provocar la devolución de un valor erróneo<br>
      <strong>Mantenimiento:</strong> Sin comentario.<br>
      <strong>Claridad:</strong> Documenta el código para indicar que se asume que la lista de monomios siempre está correctamente inicializada. La lógica de recorrido es confusa y no se separa en funciones auxiliares para mejorar la legibilidad. <br>
      <strong>Eficiencia:</strong> Sin comentario.<br>
      <em>Sugerencia Formativa:</em> Añade comprobaciones para el caso en que <code>p</code> sea NULL o <code>p->primero</code> no exista, y documenta cada paso del recorrido.
    </td>
  </tr>
  <tr>
    <td>poly_coeficiente</td>
    <td>0.5 / 1</td>
    <td>
      <strong>Funcionalidad:</strong> La función no contempla correctamente el caso en que se solicite un exponente negativo o inexistente. No se manejan adecuadamente las condiciones de salida, lo que puede provocar bucles infinitos o retornos incorrectos.<br>
      <strong>Mantenimiento:</strong> La estructura de la función no es modular y dificulta la localización de errores.<br>
      <strong>Claridad:</strong> Existe confusión entre variables locales y la utilización de punteros, lo que puede derivar en errores de acceso a memoria.<br>
      <strong>Eficiencia:</strong> Sin comentarios.<br>
      <em>Sugerencia Formativa:</em> Revisa la lógica para la búsqueda del monomio y valida la entrada de datos, asegurándote de que el bucle de recorrido tenga condiciones de terminación claras.
    </td>
  </tr>
  <tr>
    <td>poly_agregar</td>
    <td>1.0 / 2.5</td>
    <td>ejemp
      <strong>Funcionalidad:</strong> Al insertar un monomio, la función no suma correctamente coeficientes existentes y no elimina los nodos con coeficiente 0. Se realizan operaciones aritméticas sin verificar previamente el estado de la lista, lo que genera inconsistencias en la estructura.<br>
      <strong>Mantenimiento:</strong> La validación de errores en la asignación de memoria es incompleta, y el flujo se interrumpe sin notificar adecuadamente al usuario. Se recomienda crear funciones auxiliares para insertar y borrar, reduciendo la complejidad de poly_crear actual y mejorando su mantenimiento.<br>
      <strong>Claridad:</strong> Los nombres de las variables son cortos y poco descriptivos, intenta poner nombres que representen lo que almacenan.<br>
      <strong>Eficiencia:</strong> Sin comentarios.L<br>
      <em>Sugerencia Formativa:</em> Trabaja siempre pensando en la mantenibilidad del código, separando en funciones auxiliares, esto facilitará la lectura y actualización del código. 
  </tr>
  <tr>
    <td>poly_imprimir</td>
    <td>0.75 / 1.25</td>
    <td>
      <strong>Funcionalidad:</strong> La impresión del polinomio no respeta el formato indicado y omite el manejo correcto del caso en que el polinomio no existe. El recorrido de la lista no incluye condiciones de finalización adecuadas, lo que podría provocar errores si la lista está corrupta. El uso de variables de control en el bucle es inadecuado, y se accede a punteros sin verificar si son NULL.<br>
      <strong>Mantenimiento:</strong> La función es difícil de extender para soportar nuevos formatos de salida.<br>
      <strong>Claridad:</strong> Vuelves a usar nombres de una sola letra para almacenar variables, usa nombres más largos y descriptivos.<br>
      <strong>Eficiencia:</strong>Sin comentarios <br>
      <em>Sugerencia Formativa:</em> Es importante leer bien las especificaciones de los programas a desarrollar, las entrada  esperadas y las salidas son la conexión con el resto del sistema y deben respetarse.
    </td>
  </tr>
  <tr>
    <td>poly_destruir</td>
    <td>0.75 / 1.25</td>
    <td>
      <strong>Funcionalidad:</strong> La función no libera correctamente todos los nodos, dejando posibles fugas de memoria. Se produce un manejo erróneo de punteros al liberar la memoria, ya que se pierde la referencia al siguiente nodo antes de liberar el actual.<br>
      <strong>Mantenimiento:</strong> Sin comentarios.<br>
      <strong>Claridad:</strong>El código carece de comentarios y no sigue un patrón claro para la liberación de recursos. <br>
      <strong>Eficiencia:</strong>Sin comentarios. <br>
      <em>Sugerencia Formativa:</em> Las fugas de memoria son un problema común en C, asegúrate de liberar todos los nodos correctamente y documenta el proceso de liberación para facilitar la comprensión del código.
    </td>
  </tr>
  <tr>
    <td>poly_sumar</td>
    <td>1.0 / 2</td>
    <td>
      <strong>Funcionalidad:</strong> La función de suma no maneja correctamente los coeficientes cuando ambos polinomios tienen monomios con el mismo exponente; además, no se verifican los casos en que uno de los polinomios es NULL. La asignación de memoria para el polinomio resultante no está bien controlada, y la función no finaliza con <code>exit(-1)</code> en caso de fallo crítico.<br>
      <strong>Mantenimiento:</strong> La implementación resulta en una duplicación innecesaria de código, lo que complica el mantenimiento y futura extensión<br>
      <strong>Claridad:</strong> Se mezclan variables locales y punteros sin una adecuada validación, documenta su uso para facilitar la depuración.<br>
      <strong>Eficiencia:</strong> Se añade cada elemento de la lista en la lista total, esto es ineficiente ya que se hacen tantas inserciones como la suma de elementos en ambas listas. Es más eficiente ir recorriendo cada lista y sumando los que tengan el mismo exponente, así se reduce en el mejor de los casos al tamaño de la lista mayor.<br>
      <em>Sugerencia Formativa:</em> Revisa la lógica de suma para unificar el tratamiento de monomios con el mismo exponente y utiliza funciones auxiliares para gestionar la asignación y validación de memoria.
    </td>
  </tr>
</table>"""

     
            

## El siguiente método se encarga de compilar y devolver el prompt ya generado
* **Rúbrica general**: Contendrá las competencias que el docente quiere trabajar durante toda la asignatura (mantenibilidad, eficiencia, funcionalidad, etc).
* **Rúbrica específica**: Desglosará los métodos/funciones, su descripción y sus pesos sobre la calificación de la prueba.
* **Ejemplo de salida**: Tabla HTML que muestra un ejemplo de salida. Es interesante añadir los comentarios de retroalimentación que se esperan, y los que por experiencia, suele ser dónde fallan.

In [ ]:
def prompt(userCode,compilation_output):
    return f""" {rubricaGeneral()}
                {rubricaEspecifica()}
                {ejemploSalida()}
                Evalua el siguiente código del estudiante.{compilation_output}
                {userSol}"""

In [ ]:
def compilarCodigo(input_file,MODEL):
    #Modifica para copiar el fichero del estudiante con su solución al espacio de trabajo antes de copilar    
    dest = "./EspacioTrabajo/PolinomioL/polinomio.c"
    cp_command = f'cp "{input_file}" {dest}'
    subprocess.run(cp_command, shell=True)
    print("Copia realizada")

    #Comando de compilación                                                                  
    command = 'gcc principal.c PolinomioL/Polinomio.c -Wall -Wextra -Wpedantic'
    try:
        result = subprocess.run(
            command, shell=True, text=True,capture_output=True
        )
        compilation_output = "compila con ERRORES" if "error generated" in result.stdout else "compila correctamente"

    except Exception as e:
        print("An error occurred while running the compilation command:", str(e))

    userCode = ""
    with open(dest, "r", encoding="utf-8") as infile:
        for line_number, line_content in enumerate(infile, start=1):
            userCode += f"{line_content}"
    return userCode,compilation_output
    

In [ ]:
def readPrueba(input_file,MODEL):
    codigoUsario,salidaCompilacion=compilarCodigo(input_file,MODEL)
    return salidaCompilacion,f"""{rubricaGeneral()}
{rubricaEspecifica()}
{ejemploSalida()}

Evalua el siguiente código del estudiante que {salidaCompilacion}:

{codigoUsario}
    """
    

## Importante realizar una primera prueba y ver que la salida del prompt sea correcta
Localiza un examen de un estudiante y haz la prueba a generar el prompt. Luego introduce un error que no permita compilar y vuelve a generar. Así puebas que la generación sea correcta en ambos casos, cuando compila y cuando no.

In [ ]:
a=readPrueba("./prueba/ESTUDIANTE.c",MODEL_GPT)
print(a)

In [ ]:
def EvaluarPrueba(estudiante,MODEL):
    print("----------------------")
    # Construimos el mensaje para el modelo
    salidaCompilacion, prompt=readPrueba(estudiante, MODEL)
    message = [
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(
        messages=message,
        model=MODEL,
    )
    result= f"{salidaCompilacion}<br>\n" + response.choices[0].message.content
    return result

## Otra prueba
Escoge dos o tres estudiantes y observa si consigue calificar bien antes de continuar. Si observas que falta información o que has olvidado algo en alguna rúbrica, ahora es el momento de arreglarlo.


In [ ]:
modelo = MODEL_GPT
estudiante = "./pruebas/OTROESTUDIANTE.c"

print(f"Model {MODEL_GPT}")
resultado = EvaluarPrueba(estudiante, modelo)
print("Resultado obtenido")
print(resultado)

## Llegamos al paso final
Aquí se va iterando sobre todos los examenes y se solicita el LLM que genere la retroalimentación formativa. 

Observa que se genera un fichero resultados con el prompt de entrada y la salida.

La salida se guarda con extensión HTML y con el mismo nombre que tiene el fichero de pruebas.

In [ ]:
import os
import time
# Define la carpeta donde están los archivos .java
carpeta = "./pruebas/"
archivo_centralizado = "resultados.txt"
resultados_globales = []
# Función para procesar todos los archivos .java
def procesar_pruebas(carpeta, modelo):
    # Recorre todos los archivos en la carpeta
    for filename in os.listdir(carpeta):
        # Solo procesa los archivos con extensión .java
        if filename.endswith(".c"):
            print("---------------------------")
            print(filename)
            estudiante = os.path.join(carpeta, filename)

            # Llama a la función EvaluarExamen y obtén el prompt
            resultado = EvaluarPrueba(estudiante, modelo)

            resultados_globales.append(f"{estudiante}:{resultado}")
            
            # Crea el nombre del archivo de salida con extensión .txt
            archivo_resultado = os.path.splitext(estudiante)[0] + ".html"
            
            # Guarda el resultado en el archivo .txt
            with open(archivo_resultado, "w") as f:
                f.write(resultado)
                print(f"Resultado guardado en: {archivo_resultado}")
            
    with open(archivo_centralizado, "w") as f:
        f.write("\n".join(resultados_globales))
        print(f"Resultados centralizados guardados en: {archivo_centralizado}")
    

# Llamada a la función con el modelo
procesar_pruebas(carpeta, MODEL_GPT)
